<a href="https://colab.research.google.com/github/ShangLin-Wu/Speech-to-text/blob/main/speechToText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install deepspeech==0.9.3
!wget https://github.com/mozilla/DeepSpeech/releases/download/v0.9.3/deepspeech-0.9.3-models.pbmm
!wget https://github.com/mozilla/DeepSpeech/releases/download/v0.9.3/deepspeech-0.9.3-models.scorer


from deepspeech import Model
import numpy as np
import os
import wave
import json

from IPython.display import Audio

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libasound2-dev is already the newest version (1.1.3-5ubuntu0.5).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 30 not upgraded.
Need to get 184 kB of archives.
After this operation, 891 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudiocpp0 amd64 19.6.0-1 [15.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 portaudio19-dev amd64 19.6.0-1 [104 kB]
Fetched 184 kB in 1s (261 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 160980 files and directories currently installed.)
Preparing to 

In [9]:
model_file_path = 'deepspeech-0.9.3-models.pbmm'
lm_file_path = 'deepspeech-0.9.3-models.scorer'
beam_width = 100
lm_alpha = 0.6
lm_beta = 2.5

model = Model(model_file_path)
model.enableExternalScorer(lm_file_path)

model.setScorerAlphaBeta(lm_alpha, lm_beta)
model.setBeamWidth(beam_width)

stream = model.createStream()

# read wav file
def read_wav_file(filename):
    with wave.open(filename, 'rb') as w:
        rate = w.getframerate()
        frames = w.getnframes()
        buffer = w.readframes(frames)

    return buffer, rate

from google.colab import files

# read mp3 file
def transferMp3ToWav():
  uploaded = files.upload()
  for audio in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
         name=audio, length=len(uploaded[audio])))

  os.rename(audio, 'speech.mp3')
  audio = 'speech.wav'

  # convert to wav file.  
  !ffmpeg -i speech.mp3 -vn -acodec pcm_s16le -ac 1 -ar 16000 -f wav speech.wav
  #!mpg123 -w speech.wav speech.mp3

from IPython.display import clear_output

def transcribe_streaming(audio_file):
    buffer, rate = read_wav_file(audio_file)
    offset = 1024
    batch_size = 2048
    text=''

    while offset < len(buffer):
      end_offset=offset+batch_size
      chunk=buffer[offset:end_offset]
      data16 = np.frombuffer(chunk, dtype=np.int16)

      stream.feedAudioContent(data16)
      text=stream.intermediateDecode()
      #clear_output(wait=True)
      print(text)
      offset=end_offset
    return True

transferMp3ToWav()
transcribe_streaming('speech.wav')

Saving en-natureAccent.mp3 to en-natureAccent.mp3
User uploaded file "en-natureAccent.mp3" with length 72192 bytes
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis -

True

In [ ]:
# Original answer
# 'A decision has been made to retain the most valuable elements of the BBC's red button text service. We would like to thank all those who contacted us to share their views about the red button text service. Visit our About the BBC blog to read more about the decision and to find out more about changes to the red button text service.

In [ ]:
# beam_width = 100
# lm_alpha = 0.6
# lm_beta = 2.5
# offset = 1024
# batch_size = 2048
# 'a decision has been made to retain the most valuable elements of the b b c's red button text service we would like to think all those who contracted as to share their views about the red button text service this it are about the bbc blog to read more about the decision and to find out more about changes to the red button text service'


In [ ]:
# beam_width = 100
# lm_alpha = 0.3
# lm_beta = 0.18
# offset = 1024
# batch_size = 2048
# 'decision has been made to retain the most valuable elements of the bbc s red button text service we would like to thank all those who contacted as to share their views about the red button text service visit are about the bbc blog to read more about the decision and to find out more about changes to the red button tex service'


In [5]:
# beam_width = 100
# lm_alpha = 0.5
# lm_beta = 0.9
# offset = 1024
# batch_size = 2048
# 'decision has been made to retain the most valuable elements of the bbc red button text service we would like to think all those who contracted as to share their views about the red button text service visit are about the bbc blog to read more about the decision and to find out more about changes to the red button text service'